# R1CS to QAP

We take the following example : 

**out = x⁴ - 5y²x²**

```sh
v1 = x * x
v2 = v1 * v1         # x^4
v3 = -5y * y
-v2 + out = v3*v1    # -5y^2 * x^2
```

In [8]:
import numpy as np

# Python lib for modular arithmetic
import galois

In [9]:
# 1, out, x, y, v1, v2, v3
L = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, -5, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
])

O = np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, -1, 0],
])

In [10]:
x = 4
y = -2
v1 = x * x
v2 = v1 * v1         # x^4
v3 = -5*y * y
out = v3*v1 + v2    # -5y^2 * x^2

witness = np.array([1, out, x, y, v1, v2, v3])
print(witness)

var_1 = np.matmul(L, witness) * np.matmul(R, witness)
print(var_1)

var_2 = np.matmul(O, witness)
print(var_2)

assert all(np.equal(np.matmul(L, witness) * np.matmul(R, witness), np.matmul(O, witness))), "not equal"

[  1 -64   4  -2  16 256 -20]
[  16  256  -20 -320]
[  16  256  -20 -320]


### Finite Field Arithmetic Working

In [11]:
GF = galois.GF(79) # Defining the field

a = GF(70)
b = GF(10)

print(a + b)

1


In [12]:
# ================================================
# Redefining the matrices for our Modulo 79 field.
# ================================================

L = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 74, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
])

O = np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, 78, 0],
])

In [21]:
L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)

x = GF(4)
y = GF(79-2) # we are using 79 as the field size, so 79 - 2 is -2. We can't use negative number
v1 = x * x
v2 = v1 * v1         # x^4
v3 = GF(79-5)*y * y
out = v3*v1 + v2    # -5y^2 * x^2

witness = GF(np.array([1, out, x, y, v1, v2, v3]))

print("Witness [GF] : ", witness)

assert all(np.equal(np.matmul(L_galois, witness) * np.matmul(R_galois, witness), np.matmul(O_galois, witness))), "not equal"

Witness [GF] :  [ 1 15  4 77 16 19 59]


### Polynomial Interpolation in Finite Fields

In [22]:
def interpolate_column(col):
    xs = GF(np.array([1,2,3,4]))
    # print(galois.lagrange_poly(xs, col))
    return galois.lagrange_poly(xs, col)

# axis 0 is the columns. apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
U_polys = np.apply_along_axis(interpolate_column, 0, L_galois)
V_polys = np.apply_along_axis(interpolate_column, 0, R_galois)
W_polys = np.apply_along_axis(interpolate_column, 0, O_galois)

print(U_polys)
print(V_polys)
print(W_polys)

[Poly(0, GF(79)) Poly(0, GF(79)) Poly(13x^3 + 41x^2 + 22x + 4, GF(79))
 Poly(42x^3 + 22x^2 + 35x + 59, GF(79))
 Poly(40x^3 + 75x^2 + 49x + 73, GF(79)) Poly(0, GF(79))
 Poly(66x^3 + 78x^2 + 15x + 78, GF(79))]
[Poly(0, GF(79)) Poly(0, GF(79)) Poly(13x^3 + 41x^2 + 22x + 4, GF(79))
 Poly(39x^3 + 43x^2 + 72x + 4, GF(79))
 Poly(27x^3 + 74x^2 + 64x + 72, GF(79)) Poly(0, GF(79)) Poly(0, GF(79))]
[Poly(0, GF(79)) Poly(66x^3 + 78x^2 + 15x + 78, GF(79)) Poly(0, GF(79))
 Poly(0, GF(79)) Poly(13x^3 + 41x^2 + 22x + 4, GF(79))
 Poly(53x^3 + 76x^2 + 34x + 74, GF(79))
 Poly(39x^3 + 43x^2 + 72x + 4, GF(79))]


![Alt text](./assets/image-17.png)

In [24]:
from functools import reduce


def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

term_1 = inner_product_polynomials_with_witness(U_polys, witness)
term_2 = inner_product_polynomials_with_witness(V_polys, witness)
term_3 = inner_product_polynomials_with_witness(W_polys, witness)

print("Term 1 : ",term_1)
print("Term 2 : ",term_2)
print("Term 3 : ",term_3)

Term 1 :  78x^3 + 76x^2 + 28x + 59
Term 2 :  11x^3 + 77x^2 + 20x + 54
Term 3 :  3x^3 + 40x^2 + 20x + 32


In [27]:
# t = (x - 1)(x - 2)(x - 3)(x - 4)
t = galois.Poly([1, 78], field = GF) * galois.Poly([1, 77], field = GF) * galois.Poly([1, 76], field = GF) * galois.Poly([1, 75], field = GF)
print("T(x) : ", t)

h = (term_1 * term_2 - term_3) // t
print("H(x) : ", h)

# To check for remainder
assert term_1 * term_2 == term_3 + h * t, "division has a remainder"

T(x) :  x^4 + 69x^3 + 35x^2 + 29x + 24
H(x) :  68x^2 + 17x + 59
